<a href="https://colab.research.google.com/github/tomonari-masada/course2022-nlp/blob/main/07_PyTorch_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch入門 (3)
* IMDbデータセットの感情分析をPyTorchを使っておこなう。
 * 前にscikit-learnを使って同じ作業をおこなった。
* 参考資料
 * PyTorch公式のチュートリアル https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
* データは以前作ったIMDbの文書埋め込みを使う。
* sentiment analysisのもっと高度な手法については、下記リンク先を参照。
 * https://github.com/bentrevett/pytorch-sentiment-analysis

## fastTextによる文書埋め込み

### 準備

* あらかじめランタイムのタイプをGPUに設定しておこう。

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

np.random.seed(123)
torch.manual_seed(123)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

In [ ]:
!nvidia-smi

### 事前にfasttextでベクトル化されたIMDbデータを読み込む
 * 下記のリンク先にある`.npy`ファイルを、あらかじめ自分のGoogle Driveの適当な場所に置いておく。
  * https://drive.google.com/drive/folders/1wSoIzSbZ2UqGQowiVDBI20h_A3hQNbtV?usp=sharing

In [ ]:
PATH = '/content/drive/MyDrive/2022Courses/nlp/imdb/'

texts = dict()
labels = dict()
for tag in ['train', 'test']:
  with open(f'{PATH}{tag}.npy', 'rb') as f:
    texts[tag] = np.load(f)
  with open(f'{PATH}{tag}_labels.npy', 'rb') as f:
    labels[tag] = np.load(f)

In [ ]:
for tag in ['train', 'test']:
  print(texts[tag].shape)

In [ ]:
for tag in ['train', 'test']:
  texts[tag], labels[tag] = torch.tensor(texts[tag]), torch.tensor(labels[tag])

## データセットの準備

### Dataset

In [ ]:
from torch.utils.data import Dataset, random_split

class MyDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, index):
    return self.X[index], self.y[index]

train_valid = MyDataset(texts['train'], labels['train'])
test = MyDataset(texts['test'], labels['test'])

valid_size = len(train_valid) // 5
train_size = len(train_valid) - valid_size
train, valid = random_split(train_valid,
                            [train_size, valid_size],
                            generator=torch.Generator().manual_seed(42)
                            )

### DataLoader

In [ ]:
from torch.utils.data import DataLoader

# ミニバッチのサイズ
BATCH_SIZE = 100

# 訓練データだけシャッフル
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid, batch_size=BATCH_SIZE)
test_loader = DataLoader(test, batch_size=BATCH_SIZE)

## モデルの定義と学習の準備

### モデルの定義

In [ ]:
class TextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class):
    super(TextSentiment, self).__init__()
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
EMBED_DIM = texts['train'].size(1)
NUM_CLASS = len(np.unique(labels['train']))
model = TextSentiment(EMBED_DIM, NUM_CLASS).to(device)

In [ ]:
print(EMBED_DIM, NUM_CLASS)

### 損失関数と最適化アルゴリズム

* 損失関数を除いて、以下の設定はいい加減なので、自分で調整してみよう。
* schedulerの使い方は、調べてみよう。

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20,50], gamma=0.1)

## 分類器の訓練と評価

### 評価を行なう関数
* 正解率で評価する関数を定義しておく。

In [ ]:
def eval(model, criterion, loader):
  model.eval()
  
  total_loss = 0.0
  total_acc = 0.0
  total_size = 0
  for input, target in loader:
    with torch.no_grad():
      input, target = input.to(device), target.to(device)
      output = model(input)
      loss = criterion(output, target)
      total_loss += loss.item() * len(target)
      total_acc += (output.argmax(1) == target).sum().item()
      total_size += len(target)

  return total_loss / total_size, total_acc / total_size

### 訓練を行なう関数

In [ ]:
def train(model, criterion, optimizer, train_loader, valid_loader, n_epochs=100):
  model.train()

  # training loop
  for epoch in range(n_epochs):

    train_loss = 0.0
    for input, target in train_loader:
      output = model(input.to(device))
      loss = criterion(output, target.to(device))
      train_loss += loss.item() * len(target) # 表示用の集計

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

    valid_loss, valid_acc = eval(model, criterion, valid_loader)

    # logging
    print(f'epoch {epoch + 1:6d} |',
          f'train loss {train_loss / train_size:8.4f} |',
          f'valid loss {valid_loss:8.4f} | valid acc {valid_acc:8.3f}')

### 訓練と評価の実施

In [ ]:
train(model, criterion, optimizer, train_loader, valid_loader, 100)

* training lossとvalidation lossの差が大きいと、generalizeしない。
* 以下、各自試行錯誤してください。

* ハイパーパラメータのチューニングが済んだら、テストセットで評価する。



---



---



## 単語埋め込みもパラメータになっているモデル
* fasttextの単語埋め込みを使うのをやめる。
* 単語埋め込みも同時に学習することにする。

### IMDbデータセットをテキストデータとして読み直す

In [ ]:
!pip install ml_datasets

In [ ]:
from ml_datasets import imdb
train_data, test_data = imdb()

In [ ]:
train_texts, train_labels = zip(*train_data)
test_texts, test_labels = zip(*test_data)

In [ ]:
train_texts[0]

In [ ]:
train_labels[0]

### ラベルを0/1の整数に変換

In [ ]:
import numpy as np

unique_labels = np.unique(train_labels)
label_id = {}
for i, label in enumerate(unique_labels):
  label_id[label] = i

In [ ]:
train_labels = [label_id[label] for label in train_labels]
test_labels = [label_id[label] for label in test_labels]

In [ ]:
print(train_labels[:10])

### sklearnのCountVectorizerを使ってトークン化

* 語彙集合の構築

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=10, max_df=0.2)
vectorizer.fit(train_texts)

In [ ]:
vocab = vectorizer.get_feature_names_out()
print([vocab[i] for i in range(10)])

* ある単語が語彙集合に入っているかどうかは、下のようにしてチェックできる。

In [ ]:
'to' in vectorizer.vocabulary_

In [ ]:
'machine' in vectorizer.vocabulary_

* preprocessorとtokenizerの作成

In [ ]:
preprocessor = vectorizer.build_preprocessor()
tokenizer = vectorizer.build_tokenizer()

* トークン列をインデックス列に変換する関数
 * 単語のインデックスを、パディング用の単語と、未知語との２つ分、後ろにずらす。
 * テキストの長さを`max_len`に揃えるという作業も同時に行なう。

In [ ]:
PAD_IDX = 0
UNK_IDX = 1
VOCAB_SIZE = len(vocab) + 2

def encode(text, max_len=1000, padding_idx=PAD_IDX, unknown_idx=UNK_IDX):
  idx_seq = []
  for token in tokenizer(preprocessor(text)):
    if token in vectorizer.vocabulary_:
      # PAD_IDX=0とUNK_IDX=1を追加したので、通常の単語のインデックスは2つ増やす
      idx_seq.append(vectorizer.vocabulary_[token] + 2) 
    else:
      idx_seq.append(unknown_idx)
  if len(idx_seq) < max_len:
    idx_seq += [padding_idx] * (max_len - len(idx_seq))
  else:
    idx_seq = idx_seq[:max_len]
  return idx_seq

In [ ]:
print(VOCAB_SIZE)

In [ ]:
print(encode(train_texts[0]))

* バッチ単位でトークン列をインデックス列に変換する関数

In [ ]:
def batch_encode(texts):
  sequences = []
  for text in texts:
    sequences.append(encode(text))
  return torch.Tensor(sequences)

### Dataset

In [ ]:
from torch.utils.data import Dataset, random_split

class MyTextDataset(Dataset):
  def __init__(self, texts, labels):
    self.texts = texts
    self.labels = labels

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, index):
    return self.texts[index], self.labels[index]

train_valid_set = MyTextDataset(train_texts, train_labels)
test_set = MyTextDataset(test_texts, test_labels)

valid_size = len(train_valid_set) // 5
train_size = len(train_valid_set) - valid_size
train_set, valid_set = random_split(train_valid_set,
                                    [train_size, valid_size],
                                    generator=torch.Generator().manual_seed(42)
                                    )

### DataLoader
* ミニバッチのテキストをインデックス列へ変換するcollation用の関数も定義する。

In [ ]:
def collate_fn(batch):
  batch_texts, batch_labels = zip(*batch)
  batch_texts = batch_encode(batch_texts)
  return batch_texts.type(torch.LongTensor), torch.LongTensor(batch_labels)

In [ ]:
from torch.utils.data import DataLoader

# ミニバッチのサイズ
BATCH_SIZE = 100

# 訓練データだけシャッフル
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, collate_fn=collate_fn)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, collate_fn=collate_fn)

### モデルの定義
* `nn.Embedding`を使う。
 * 語彙サイズ、埋め込みの次元数、パディング用の特殊なトークンのインデックスを指定する。
 * パディング用トークンのembeddingはゼロベクトルになる。

In [ ]:
class EmbeddedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx=PAD_IDX):
    super(EmbeddedTextSentiment, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)
    self.dropout = nn.Dropout()

  def forward(self, text):
    embedded = self.dropout(self.embed(text))
    x = embedded.mean(1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

* モデルのインスタンスを作成

In [ ]:
EMBED_DIM = 300
NUM_CLASS = len(np.unique(train_labels))
model = EmbeddedTextSentiment(EMBED_DIM, NUM_CLASS, VOCAB_SIZE, PAD_IDX).to(device)

### 損失関数と最適化アルゴリズム

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20,50], gamma=0.1)

### 学習の実行

In [ ]:
train(model, criterion, optimizer, train_loader, valid_loader, 100)

In [ ]:
loss, acc = eval(model, criterion, train_loader)
print(f'train loss {loss:8.4f} | train acc {acc:8.3f}')

* 検証セット上での評価値でチューニングしてから、テストセットで最終評価。

In [ ]:
loss, acc = eval(model, criterion, test_loader)
print(f'test loss {loss:8.4f} | test acc {acc:8.3f}')

# 課題
* モデルやoptimizerやschedulerを変更して、validation setを使ってチューニングしよう。
* 最後に、自分で選択した設定を使って、test set上で評価しよう。

In [ ]:
loss, acc = eval(model, criterion, test_loader)
print(f'test loss {loss:8.4f} | test acc {acc:8.3f}')